In [1]:
import librosa
import nemo.collections.asr as nemo_asr
from IPython.display import Audio, display
from plotly import graph_objects as go
import numpy as np
import pandas as pd
import gzip
import os, shutil, wget
import ctc_decoders
import copy
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import cdl_asr

[NeMo W 2023-03-28 08:59:07 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-03-28 08:59:08 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [2]:
import transformers
transformers.__version__

'4.27.1'

In [3]:
#nemo_asr.models.EncDecCTCModel.list_available_models()

In [4]:
asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name='QuartzNet15x5Base-En', strict=False)

[NeMo I 2023-03-28 08:59:08 cloud:56] Found existing object /home/stephan/.cache/torch/NeMo/NeMo_1.16.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2023-03-28 08:59:08 cloud:62] Re-using file from: /home/stephan/.cache/torch/NeMo/NeMo_1.16.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2023-03-28 08:59:08 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-03-28 08:59:09 features:286] PADDING: 16
[NeMo I 2023-03-28 08:59:09 audio_preprocessing:513] Numba CUDA SpecAugment kernel is being used
[NeMo I 2023-03-28 08:59:10 save_restore_connector:247] Model EncDecCTCModel was successfully restored from /home/stephan/.cache/torch/NeMo/NeMo_1.16.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


In [5]:
# Download audio sample which we'll try
test_audio_filename = '1919-142785-0028.wav'
!wget https://dldata-public.s3.us-east-2.amazonaws.com/1919-142785-0028.wav

# load audio signal with librosa
signal, sample_rate = librosa.load(test_audio_filename, sr=None)

# display audio player for the signal
display(Audio(data=signal, rate=sample_rate))

--2023-03-28 08:59:10--  https://dldata-public.s3.us-east-2.amazonaws.com/1919-142785-0028.wav
Resolving dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)... 52.219.93.82
Connecting to dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)|52.219.93.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 165164 (161K) [audio/wav]
Saving to: ‘1919-142785-0028.wav.15’

1919-142785-0028.wa 100%[===================>] 161.29K  --.-KB/s    in 0.1s    

2023-03-28 08:59:11 (1.23 MB/s) - ‘1919-142785-0028.wav.15’ saved [165164/165164]



# Beam Search with N-gram model

In [6]:
lm_path = 'lowercase_3-gram.pruned.1e-7.arpa'

In [7]:
cdl_asr.load_default_lm(lm_path)

LM model already exists at lowercase_3-gram.pruned.1e-7.arpa


In [8]:
cdl_asr.transcribe_with_ngram_model([test_audio_filename], lm_path, asr_model)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcript: "boil them before they are put into the soup or other dish they may be intended for"


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Loading the LM will be faster if you build a binary file.
Reading lowercase_3-gram.pruned.1e-7.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


,hypothesis,score
0,boil them before they are put into the soup or...,-53.302773
1,boil them before they are put into the soup or...,-62.710724
2,boil them before they are put in to the soup o...,-63.176544
3,boil them before they are put into the soup or...,-63.340088
4,broil them before they are put into the soup o...,-63.944374
...,...,...
251,boil them before they are put into the soup or...,-2071.341553
252,boil them before they are put in to the soup o...,-2071.398193
253,boil them before they are put into the soup or...,-2071.561523
254,boil them before they are put into the soup or...,-2071.590576


# Pseudo Log Likelihood Rescoring with BERT

In [9]:
import imp
imp.reload(cdl_asr)

[NeMo W 2023-03-28 08:59:20 nemo_logging:349] /tmp/ipykernel_2048782/2921217217.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
      import imp
    


<module 'cdl_asr' from '/home/stephan/notebooks/cdl-asr/cdl_asr.py'>

In [12]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bertMaskedLM = BertForMaskedLM.from_pretrained('bert-base-uncased')
bertMaskedLM.eval()
bertMaskedLM.to('cuda')  # if you have gpu
vocab =  np.array(bert_tokenizer.convert_ids_to_tokens(range(bert_tokenizer.vocab_size)))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
%%time
cdl_asr.transcribe_with_neural_rescoring([test_audio_filename], lm_path, asr_model, bertMaskedLM, vocab, bert_tokenizer)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcript: "boil them before they are put into the soup or other dish they may be intended for"


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Loading the LM will be faster if you build a binary file.
Reading lowercase_3-gram.pruned.1e-7.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


CPU times: user 1min 2s, sys: 1.24 s, total: 1min 3s
Wall time: 1min 2s


,hypothesis,score
73,boil them before they are put into a soup or o...,19.020205
0,boil them before they are put into the soup or...,19.411890
1,boil them before they are put into the soup or...,19.411890
44,boil them before they are put into the soup or...,20.725145
2,boil them before they are put in to the soup o...,21.145136
...,...,...
161,broil them before they are put into the soup o...,64.869087
156,boil them before they are put into the soup or...,65.453827
157,boil them before they are put into the soup or...,65.926010
247,boil them before they are put into the soup or...,67.916664


# Todos

[X] pandafy the outputs and get the value counts  
[X] is the beam search only looking at the last thing  
[X] do PPL rescoring using one of my models  
-----[X] Figure out why BERT results are wrong for these inputs   
----------[X] Is it a difference in the versioning? yes, it starts working if we load it twice  
[X] work on optimizing PPL rescoring speed -- 64 in 14s for a long sentence 
-----[X] Is there a way to finetune on BERT and then do Salazar-style one-shot utterance probability  estimation    
[wontfix] figure out how to rescore up to the elbow -- fast enough for now  
-----[wontfix] reduce the number of hypotheses -- but that is less fun  
[wontfix] handle when there is a trailing space -- let this get handled in the rescoring  


[ ] librarify this  
[ ] switch out the ngram model with one from CDL  
[ ] switch out the CHILDES model   



[ ] interpolate the weights of the initial system and the rescoring    
-----[ ] think about how this should handle many different values of the weights   
[ ] make it shut up about the widget not found problem   



[ ] Make the test harness      
-----[ ] think about how to extract these from the Providence dataset. Sophia has a timestamp getter?  
-----[ ] want utterance level WERs  
-----[ ] Find the test notebook that I did for Alex Cristia -- YobiYoba?
[ ] implement the cross in models. Be careful about overfitting
[ ] performance on CDL subsetted words vs. everything else  
[ ] consider doing the same thing with the Whisper model and a larger beam in decoding  

[ ] diarization based on language model. Acoustics and the lexical signal 